# Counter Strategy
## 1. Graph Construction

In [1]:
import graph as g
with open("data/test01.in",'r') as file:
    V,E=map(int,file.readline().rstrip().split())
    G=g.LabeledGraph(V)
    for k in range(E):
        G.addEdge(map(int,file.readline().rstrip().split()))
    psi=list(map(int,file.readline().rstrip().split()))
G=g.read_from_text_file("data/test01.in",graph_type="auto")
import graph_visualisation as gv
gv.visualise_graph(G)

GraphWidget(layout=Layout(height='500px', width='100%'))

## 2. Counter Strategy Construction
- Let $\psi:V\rightarrow V$ be the strategy of player $1$, with $\psi\subseteq E$
- Suppose that the second player has knowledge of the first player's strategy

As the first player's strategy is known, the second player can transform this game to an instance of path minimisation on the following graph:
$$
G'=(V,E',L') \quad \text{with:}\space \begin{cases}
E'=\{(u,v),\quad (\psi(u),v)\in E\} \\
L'(u,v)= L(u,\psi(u))+L(\psi(u),v) \quad \forall (u,v)\in E
\end{cases}
$$
In fact:
- The first player will lose if and only if there exists a **negative cycle** on $G'$
- The first player will win if and only if all cycles of $G'$ are positive

We will define a cycle $\mathcal{C}$ as a sequence $(u_0,\dots,u_n)$ with:
- $n\in\mathbb{N}^*$
- $\forall i\in \{1,\dots,n\},\quad (u_{i-1},u_i)\in E'$
- $u_n=u_0$
- $\forall i,j\in \{1,\dots,n\},u_i\neq u_j$

The length of the cycle $\lvert \mathcal{C}\rvert$ is defined as:
$$
\lvert \mathcal{C}\rvert=n
$$

Let $\mathscr{C}$ be the set of cycles of the graph $G',$ our problem is to minimise:
$$
H(\mathcal{C})=\frac{1}{\lvert C \rvert } \sum_{k=1}^{n}L(u_{i-1},u_{i})
$$

### 2.1 Graph $E'$


In [28]:
import networkx as nx
G2=g.LabeledGraph(G.V)
strategyCost=[-1 for u in range(G.V)]
for u in range(G.V):
    for v,L in G.adjacencyList[u]:
        if v==psi[u]:
            strategyCost[u]=L
for u in range(G.V):
    for v,L in G.adjacencyList[psi[u]]:
        G2.addEdge((u,v,L+strategyCost[u]))
gv.GraphWidget(graph=G2.as_networkx())

GraphWidget(layout=Layout(height='500px', width='100%'))

### 2.2 Bellman-Ford

In [23]:
import networkx as nx
nx.find_negative_cycle(G2.as_networkx(),0)

[2, 7, 2]

In [75]:
g.counterStrategyBellmanFord(G,psi,method="bellman-ford")

[1, 2, 3, 7, 5, 6, 1, 1]

### 2.3 Counter Strategy

In [83]:
import numpy as np
FW=nx.floyd_warshall_numpy(G2.as_networkx())
n=FW.shape[0]
FW

array([[  0.,   6.,   5.,  inf,  inf,  -1.,   4.,  inf],
       [ inf,  -4.,  -5.,  inf,  inf, -11.,  -6.,  inf],
       [ inf,   0.,  -1.,  inf,  inf,  -7.,  -2.,  inf],
       [ inf,  inf,  inf,   0.,  -2.,  inf,  inf,  inf],
       [ inf,  inf,  inf,   2.,   0.,  inf,  inf,  inf],
       [ inf,   1.,   0.,  inf,  inf,  -6.,  -1.,  inf],
       [ inf,  -2.,  -3.,  inf,  inf,  -9.,  -4.,  inf],
       [ inf,  inf,  inf,   0.,  -2.,  inf,  inf,   0.]])

In [92]:
for i in range(n):
    for j in range(n):
        for k in range(n):
            if FW[k,k]<0 and FW[i,k]<np.inf and FW[k,j]<np.inf:
                FW[i,j]=-np.inf
FW

array([[  0., -inf, -inf,  inf,  inf, -inf, -inf,  inf],
       [ inf, -inf, -inf,  inf,  inf, -inf, -inf,  inf],
       [ inf, -inf, -inf,  inf,  inf, -inf, -inf,  inf],
       [ inf,  inf,  inf,   0.,  -2.,  inf,  inf,  inf],
       [ inf,  inf,  inf,   2.,   0.,  inf,  inf,  inf],
       [ inf, -inf, -inf,  inf,  inf, -inf, -inf,  inf],
       [ inf, -inf, -inf,  inf,  inf, -inf, -inf,  inf],
       [ inf,  inf,  inf,   0.,  -2.,  inf,  inf,   0.]])

In [95]:
FWM=np.full([n,n],np.inf)
G2NX=G2.as_networkx()
for i in range(n):
    FWM[i,i]=0
for u,v in G2NX.edges:
    FWM[u,v]=G2NX.edges[u,v]["weight"]
FWM

array([[ 0., inf, 10., inf,  9., inf, inf, inf],
       [inf,  0., inf, -2., inf, inf, inf, inf],
       [inf, inf,  0., inf, inf, inf, -2., -7.],
       [inf,  2., inf,  0., inf, inf, inf, inf],
       [inf, inf, inf, inf,  0., inf,  0., -3.],
       [inf,  0., inf, inf, inf,  0., inf, inf],
       [inf, inf,  2., inf,  1., inf,  0., inf],
       [inf, inf,  5., inf,  4., inf, inf,  0.]])

In [96]:
for k in range(n):
    for i in range(n):
        for j in range(n):
            if FWM[i,k] < np.inf and FWM[k,j] < np.inf:
                FWM[i,j]=np.minimum(FWM[i,j],FWM[i,k]+FWM[k,j])
FWM

array([[ 0., inf,  8., inf,  7., inf,  6.,  1.],
       [inf,  0., inf, -2., inf, inf, inf, inf],
       [inf, inf, -2., inf, -3., inf, -4., -9.],
       [inf,  2., inf,  0., inf, inf, inf, inf],
       [inf, inf,  0., inf, -1., inf, -2., -7.],
       [inf,  0., inf, -2., inf,  0., inf, inf],
       [inf, inf,  0., inf, -1., inf, -2., -7.],
       [inf, inf,  3., inf,  2., inf,  1., -4.]])